#LSTM & Word2Vec
โค้ดส่วนนี้ปรับมาจาก https://kobkrit.com/%E0%B9%81%E0%B8%88%E0%B8%81%E0%B9%84%E0%B8%9A%E0%B9%80%E0%B8%9A%E0%B8%B4%E0%B9%89%E0%B8%A5-%E0%B8%A7%E0%B8%B4%E0%B8%98%E0%B8%B5%E0%B8%81%E0%B8%B2%E0%B8%A3-finetune-bert-roberta-wangchanberta-%E0%B8%AA%E0%B8%B3%E0%B8%AB%E0%B8%A3%E0%B8%B1%E0%B8%9A%E0%B8%87%E0%B8%B2%E0%B8%99-nlp-%E0%B8%A0%E0%B8%B2%E0%B8%A9%E0%B8%B2%E0%B9%84%E0%B8%97%E0%B8%A2%E0%B9%81%E0%B8%9A%E0%B8%9A%E0%B8%87%E0%B9%88%E0%B8%B2%E0%B8%A2-1fbbaac7c905

## Preparing VM

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
# Command line tool for monitoring VM
! apt-get -qq install htop -y

#นำเข้าข้อมูล

In [ ]:
import pandas as pd
train= pd.read_csv('/content/train.csv')
test= pd.read_csv('/content/test.csv')

In [ ]:
train

In [ ]:
test

## Word2Vec

In [ ]:
!pip install pythainlp python-crfsuite

In [ ]:
from pythainlp import word_vector
import numpy as np


In [ ]:
wv = word_vector.WordVector()

In [ ]:
####  Word to Vec object
word2vec = wv.get_model()

In [ ]:
def get_average_vector(cleaned_query):
    sum_query = None
    count=0
    for query in cleaned_query:
        try:
            if (sum_query is None):
                sum_query = np.copy(word2vec[query])
            else:
                sum_query += word2vec[query]
            count+=1
            print("Word "+query+ " found!")
        except:
            print("Word "+query+" is not found!")
    if (sum_query is not None):
        average_query = sum_query/count
    else:
        average_query = np.zeros((300,))
    return average_query

average_query = get_average_vector(["ทดสอบ","ดู","นะ","ครับ"])
print(average_query)

In [ ]:
from functools import reduce

# Make average vectors of each class
avg_w = {}
for i in range(3):
  avg_w[i] = reduce(lambda x,y: x+y, train[train["sentiment"]==i][:100]['wordseged_text'])

In [ ]:
avg_v = {}
for i in range(3):
  print(i, len(avg_w[i]))
  avg_v[i] = get_average_vector(avg_w[i])

In [ ]:
def cosine_sim(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
qv = get_average_vector(['ประธาน', 'ไม่', 'แข็งแรง', 'พอ', 'คือ', 'ขี้กลัว', 'ทำไม', 'ไม่', 'ใช้อำนาจ', 'ประธาน', 'สภา', 'เลย', 'โหวต', 'ทำไม'])
qv

In [ ]:
for i in range(3):
    print(i, cosine_sim(qv, avg_v[i]))

## LSTM

In [ ]:
! pip install pythainlp python-crfsuite

In [ ]:
import matplotlib.pyplot as plt
train_t = train.copy()
temp = train_t.wordseged_text.apply(lambda x : len(x)).values
plt.hist(temp)

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
#from keras.layers.embeddings import Embedding
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(23)
from pythainlp import word_vector
import numpy as np
#word2vec = word_vector.get_model()


In [ ]:
print("Vocab in Thai2Vec:",len(word2vec.key_to_index))

In [ ]:
import numpy as np

def sentences_to_indices(X, word2vec, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()`.

    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word2vec -- a trained Word2Vec model from gensim
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this.

    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """

    m = X.shape[0]                                   # number of training examples

    # Initialize X_indices as a numpy matrix of zeros and the correct shape
    X_indices = np.zeros((m, max_len))

    for i in range(m):                               # loop over training examples

        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        # print(X)
        # print(len(X[i].lower().split()))
        sentence_words = X[i].lower().split()[:max_len]

        # Initialize j to 0
        j = 0
        try:
        # Loop over the words of sentence_words
          for w in sentence_words:
              # Set the (i,j)th entry of X_indices to the index of the correct word.

                if w in word2vec.key_to_index:
                    X_indices[i, j] = word2vec.key_to_index[w]
                    # Increment j to j + 1
                    j += 1
        except:
              print('key error: ', w)


    return X_indices


In [ ]:
'ฏ' in word2vec.key_to_index

In [ ]:
X1 = np.array(["สวัสดี ครับ ผม", "ทดสอบ ทดสอบ ทดสอบ", "เรา มา ทดสอบ กัน","8"])
X1_indices = sentences_to_indices(X1, word2vec, max_len = 10)
print("X1 =", X1)
print("X1_indices =", X1_indices)

In [ ]:
def convert_to_one_hot(y, maxLen):
    max_value = max(y)
    min_value = min(y)
    length = len(y)
    one_hot = np.zeros((length, (max_value - min_value + 1)))
    one_hot[np.arange(length), y] = 1
    return one_hot

In [ ]:
word2vec["สวัสดี"]

In [ ]:
 word2vec.key_to_index.items()

In [ ]:
word2vec["ประชาชน"].shape[0]

In [ ]:
vocab_len = len(word2vec.key_to_index) + 1
emb_dim = word2vec["สวัสดี"].shape[0]
emb_matrix = np.zeros((vocab_len, emb_dim))

In [ ]:
for word, index in word2vec.key_to_index.items():
      #print(word, index)
      emb_matrix[index, :] = word2vec[word]

In [ ]:
def pretrained_embedding_layer(word2vec):
    """
    Creates a Keras Embedding() layer and loads in pre-trained PyThaiNLP word_vector 300-dimensional vectors.

    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """

    vocab_len = len(word2vec.key_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word2vec["ประชาชน"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)

    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))

    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word2vec.key_to_index.items():
        # print(word)
        emb_matrix[index, :] = word2vec[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False.
    embedding_layer = Embedding(vocab_len, emb_dim)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))

    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])

    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(word2vec)
print("weights[0][1] =", embedding_layer.get_weights()[0][1])
# print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

In [ ]:
def text_clasifier_lstm(input_shape, word2vec):
    """
    Function creating the Emojify-v2 model's graph.

    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """

    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape=input_shape, dtype=np.int32)

    # Create the embedding layer pretrained with Word2Vec Vectors
    embedding_layer =  pretrained_embedding_layer(word2vec)

    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)

    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.1
    X = Dropout(0.1)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128)(X)
    # Add dropout with a probability of 0.1
    X = Dropout(0.1)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3, activation='softmax')(X)
    # Add a softmax activation
    X = Activation('softmax')(X)

    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices, X)

    return model

In [ ]:
model = text_clasifier_lstm((128,), word2vec)
model.summary()

In [ ]:
!pwd

In [ ]:
%cd /content/drive/MyDrive/Student_Project/Vote_check

In [ ]:
import tensorflow as tf
model = text_clasifier_lstm((128,), word2vec)
model.summary()
# optimizer = optim.Adam(first_model.parameters(),lr=0.1e-3,weight_decay=1e-3)
model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])
X_train_indices = sentences_to_indices(train['wordseged_space_text'].values, word2vec, 128)
Y_train_oh = convert_to_one_hot((train['sentiment']), 3)
# ถ้า X_train_indices และ Y_train_oh เป็นข้อมูลการฝึกของคุณ
history = model.fit(X_train_indices, Y_train_oh, epochs=500, batch_size=32, shuffle=True, validation_split=0.2)
# บันทึกโมเดลลง disk
model.save('my_model4.h5')

#filehandler = open('model4.pickle', 'wb')
#pickle.dump(model, filehandler)


In [ ]:
import pickle
filehandler = open('history.pickle', 'wb')
pickle.dump(history, filehandler)


In [ ]:
filehandler = open('history.pickle', 'rb')
loaded_history = pickle.load(filehandler)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd


def plot_learningCurve(histoty):

  # Extract loss and accuracy values from the training history
  train_loss_values = histoty.history['loss']
  train_accuracy_values = histoty.history['accuracy']
  val_loss_values = histoty.history['val_loss']
  val_accuracy_values = histoty.history['val_accuracy']

  # Plot training and validation loss
  plt.figure(figsize=(8, 8))

  # plt.subplot(1, 2, 1)
  plt.plot(train_loss_values, label='Training Loss')
  plt.plot(val_loss_values, label='Validation Loss')
  plt.title('Model Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()
  # Plot training and validation accuracy
  # plt.subplot(1, 2, 2)
  plt.figure(figsize=(8, 8))
  plt.plot(train_accuracy_values, label='Training Accuracy')
  plt.plot(val_accuracy_values, label='Validation Accuracy')
  plt.title('Model Accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend()

  plt.show()



def compute_Confusion(test,model):

  test2 = np.array(test['wordseged_space_text'])
  # Assuming you have test data
  X_test_indices = sentences_to_indices(test2, word2vec, 128)

  # One-hot encode the 'sentiment' column
  Y_test_oh = pd.get_dummies(test['sentiment']).values

  # Make predictions on the test set
  Y_pred = model.predict(X_test_indices)

  # Convert one-hot encoded predictions and true labels back to class labels
  Y_pred_classes = np.argmax(Y_pred, axis=1)
  Y_true_classes = np.argmax(Y_test_oh, axis=1)
  rp02 =( Y_true_classes == 0) & (Y_pred_classes == 2)
  #print(test2[rp02])
  # Confusion Matrix
  conf_matrix = confusion_matrix(Y_true_classes, Y_pred_classes)
  print("Confusion Matrix:")
  print(conf_matrix)

  # Classification Report with Precision, Recall, and F1-Score
  class_report = classification_report(Y_true_classes, Y_pred_classes)
  print("\nClassification Report:")
  print(class_report)

compute_Confusion(test, model)
plot_learningCurve(history)

In [ ]:
import matplotlib.pyplot as plt

# Extract loss and accuracy values from the training history
train_loss_values = history.history['loss']
train_accuracy_values = history.history['accuracy']
val_loss_values = history.history['val_loss']
val_accuracy_values = history.history['val_accuracy']

# Plot training and validation loss
plt.figure(figsize=(8, 8))

# plt.subplot(1, 2, 1)
plt.plot(train_loss_values, label='Training Loss')
plt.plot(val_loss_values, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
# Plot training and validation accuracy
# plt.subplot(1, 2, 2)
plt.figure(figsize=(8, 8))
plt.plot(train_accuracy_values, label='Training Accuracy')
plt.plot(val_accuracy_values, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In [ ]:
# Extract loss and accuracy values from the training history
train_loss_values = history.history['loss']
train_accuracy_values = history.history['accuracy']
val_loss_values = history.history['val_loss']
val_accuracy_values = history.history['val_accuracy']
# Plot training loss
plt.figure(figsize=(8, 8))
plt.plot(train_loss_values)
plt.title('Model Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Plot training accuracy
plt.figure(figsize=(8, 8))
plt.plot(val_loss_values)
plt.title('Model Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
X_train_indices

In [ ]:
preds = model.predict(X_train_indices[:10])
preds

In [ ]:
word2vec

In [ ]:
test2 = np.array(test['wordseged_space_text'][101:102])
sentences_to_indices(test2, word2vec, 128)

In [ ]:
# Assuming Y_test_oh is a DataFrame
Y_test_oh = convert_to_one_hot(test['sentiment'], 3)
print("One-Hot Encoded Labels:")
print(Y_test_oh)

# Assuming test['sentiment'] is the original sentiment labels
print("Original Sentiment Labels:")
print(test['sentiment'].head())


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
test2 = np.array(test['wordseged_space_text'])
# Assuming you have test data
X_test_indices = sentences_to_indices(test2, word2vec, 128)

# One-hot encode the 'sentiment' column
Y_test_oh = pd.get_dummies(test['sentiment']).values

# Make predictions on the test set
Y_pred = model.predict(X_test_indices)

# Convert one-hot encoded predictions and true labels back to class labels
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true_classes = np.argmax(Y_test_oh, axis=1)
rp02 =( Y_true_classes == 0) & (Y_pred_classes == 2)
print(test2[rp02])
# Confusion Matrix
conf_matrix = confusion_matrix(Y_true_classes, Y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report with Precision, Recall, and F1-Score
class_report = classification_report(Y_true_classes, Y_pred_classes)
print("\nClassification Report:")
print(class_report)


In [ ]:
np.argmax(preds, axis=1)